In [1]:
import numpy as np

from Solver import mesh_less, crank_nicolson_fd
from Utilz import F_v0, F_v0_x, diff, eval_F

import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.gridspec as gridspec

In [2]:
def mu(x, t, prms):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    v0 = prms[3]
    v1 = prms[4]
    tau = prms[5]
    
    v0_hat = -T*v0/a
    v1_hat = -T*v1/a
    tau_hat = tau/T
    
    return v0_hat + v1_hat*(1-t)/(1-t+tau_hat)

In [3]:
def g(x, t, prms, max_k=20):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    v0 = prms[3]
    v1 = prms[4]
    tau = prms[5]
    
    kappa = (T*sigma**2)/(2*a**2)

    if t == 0:
        return 0
    elif np.isclose(mu(x, t, prms) - mu(0, 0, prms), 0):
        return 0
    else:
        return (mu(x, t, prms) - mu(0, 0, prms)) * F_v0_x(x, t, kappa, mu(0, 0, prms), max_k)

# Error Estimation:

In [4]:
sigma = 1
a = 1.3
v0 = 1
v1 = 0.4
tau = 0.7

M_ml_1 = 6
M_fd = M_ml_1

time_step = 6

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5

for T in [.2, .4, .6, .8, 1]:
    prms = [T, sigma, a, v0, v1, tau]
    kappa = (T*sigma**2)/(2*a**2)
    
    ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.8)
    ml_3 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=1.2)
    
    fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
    print('Exact solution ...')
    fd_ex = fd_ex = np.load('_exact/exact_HD_{}.npy'.format(int(T*10)), allow_pickle=True)[()]
    
    exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
    fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')
    
    fd_err = np.sqrt(np.sum([(fdp(x) - exact(x))**2 for x in xx]))
    
    err_ml_1 = np.sqrt(np.sum([np.abs(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
    err_ml_2 = np.sqrt(np.sum([np.abs(eval_F(ml_2, x) - exact(x))**2 for x in xx]))
    err_ml_3 = np.sqrt(np.sum([np.abs(eval_F(ml_3, x) - exact(x))**2 for x in xx]))

    print('T = {}, ML_1_ERR = {}, ML_2_ERR = {}, and ML_3_ERR = {}'.format(T, err_ml_1, err_ml_2, err_ml_3))
    print('FD_ERR = {},'.format(fd_err))
    print("\t\t\t\t....................................")

Exact solution ...
T = 0.2, ML_1_ERR = 0.00011235639737518654, ML_2_ERR = 9.102247850074414e-05, and ML_3_ERR = 7.081719841184924e-05
FD_ERR = 0.0005675220866270885,
				....................................
Exact solution ...
T = 0.4, ML_1_ERR = 0.00019117552253662288, ML_2_ERR = 0.0002260571968693458, and ML_3_ERR = 0.0003011379966073432
FD_ERR = 0.0011356489814787023,
				....................................
Exact solution ...
T = 0.6, ML_1_ERR = 0.00015370324680035981, ML_2_ERR = 0.00023402297965958695, and ML_3_ERR = 0.00039271499090047507
FD_ERR = 0.0012718076067341282,
				....................................
Exact solution ...
T = 0.8, ML_1_ERR = 6.568187222805698e-05, ML_2_ERR = 0.00015863770706029103, and ML_3_ERR = 0.0003630729490389278
FD_ERR = 0.001281434991396868,
				....................................
Exact solution ...
T = 1, ML_1_ERR = 8.43549191536532e-05, ML_2_ERR = 0.00010912614570762234, and ML_3_ERR = 0.0003117336154616895
FD_ERR = 0.0012459273460854832,
				....

In [55]:
sigma = 1
a = 1.3
v0 = 1
v1 = 0.4
tau = 0.7

M_ml_1 = 6
M_fd = M_ml_1

time_step = 6

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5
T = 1
prms = [T, sigma, a, v0, v1, tau]
kappa = (T*sigma**2)/(2*a**2)

m1 = 5
m2 = 6

ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=m1, N=time_step, max_k=max_k, eps=0.4)
ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=m2, N=time_step, max_k=max_k, eps=0.4)

fd_1 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m1, N=time_step, max_k=max_k)
fd_2 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m2, N=time_step, max_k=max_k)
print('Exact solution ...')
fd_ex = fd_ex = np.load('_exact/exact_HD_{}.npy'.format(int(T*10)), allow_pickle=True)[()]

exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
fdp_1 = interp1d(fd_1['X'], np.concatenate((np.array([0]), fd_1['sol'], np.array([0]))), 'quadratic')
fdp_2 = interp1d(fd_2['X'], np.concatenate((np.array([0]), fd_2['sol'], np.array([0]))), 'quadratic')

fd_err_1 = np.sqrt(np.sum([(fdp_1(x) - exact(x))**2 for x in xx]))
fd_err_2 = np.sqrt(np.sum([(fdp_2(x) - exact(x))**2 for x in xx]))

err_ml_1 = np.sqrt(np.sum([np.abs(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
err_ml_2 = np.sqrt(np.sum([np.abs(eval_F(ml_2, x) - exact(x))**2 for x in xx]))

Exact solution ...


In [56]:
fd_err_1, fd_err_2, np.log2(fd_err_1/fd_err_2)


(0.0021072915806714847, 0.0012459273460854832, 0.758170007314292)

In [57]:
err_ml_1, err_ml_2, np.log2(err_ml_1/err_ml_2)

(0.0003338476875747724, 8.43549191536532e-05, 1.9846459405312353)

# Time Evaluation:

In [4]:
import timeit

In [35]:
sigma = 1
a = 1.3
v0 = 1
v1 = 0.4
tau = 0.7

M_ml_1 = 10
M_fd = M_ml_1

time_step = 10

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5

start = timeit.default_timer()

for _ in range(10):
    for T in np.arange(0, 1+1/5, 1/5)[1:]:
        prms = [T, sigma, a, v0, v1, tau]
        kappa = (T*sigma**2)/(2*a**2)

#         ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    
        fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
        fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')


stop = timeit.default_timer()

print('Time: ', (stop - start)/10)

Time:  1.2610864458001743


In [13]:
np.arange(0, 1+1/10, 1/10)[1:]

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])